<a href="https://colab.research.google.com/github/Abhi23run/Uncertainty_Quantification_LLMs/blob/main/Uncertainty_Quantification_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
dir_path = '/content/drive/MyDrive/Uncertainty_Quantification'
os.chdir(dir_path)

In [5]:
%ls

data_normal.csv             Sample_data_creation_stress_index.ipynb
data_with_stress_index.csv  Uncertainty_Quantification_llama.ipynb


In [6]:
%%capture
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install evaluate
!pip install -qqq trl==0.7.1
!pip install torch

In [7]:
import torch
import time
import evaluate
import pandas as pd
import numpy as np
from datasets import Dataset, load_dataset
import random
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [8]:
model_id =  "NousResearch/Llama-2-7b-hf"
# model_id = "meta-llama/Llama-2-13b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [9]:
from transformers import StoppingCriteria
class EosListStoppingCriteria(StoppingCriteria):
    def __init__(self, eos_sequence = [319,350,360,315]):
        self.eos_sequence = eos_sequence

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        last_ids = input_ids[:,-len(self.eos_sequence):].tolist()
        return self.eos_sequence in last_ids

In [10]:
import warnings
warnings.filterwarnings('ignore')

prompt = f"""
###Instruction: Act as a science expert and help in answering the following multiple choice question. The answer should be one of the option choices.
Follow the pattern from the first example and answer the second question accordingly.

###Question:
This is a question from high school biology.A piece of potato is dropped into a beaker of pure water.
Which of the following describes the activity after the potato is immersed into the water?
(A) Water moves from the potato into the surrounding water.
(B) Water moves from the surrounding water into the potato.
(C) Potato cells plasmolyze.
(D) Solutes in the water move into the potato.

###Answer:
B

###Question:
You are the world’s best expert in high school biology. From the solubility rules, which of the following is true?
(A) All hydroxides are soluble
(B) All sulfates are soluble
(C) All chlorides, bromides, and iodides are soluble
(D) All ammonium-containing compounds are soluble

###Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        temperature=0.2,
        do_sample=True,
        max_new_tokens=8,
        stopping_criteria = [EosListStoppingCriteria()]
    )[0],
    skip_special_tokens=False
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
<s> 
###Instruction: Act as a science expert and help in answering the following multiple choice question. The answer should be one of the option choices. 
Follow the pattern from the first example and answer the second question accordingly.

###Question:
This is a question from high school biology.A piece of potato is dropped into a beaker of pure water. 
Which of the following describes the activity after the potato is immersed into the water?
(A) Water moves from the potato into the surrounding water.
(B) Water moves from the surrounding water into the potato.
(C) Potato cells plasmolyze.
(D) Solutes in the water move into the potato.

###Answer:
B

###Question:
You are the world’s best expert in high school biology. From the solubility rules, which of the following is true?
(A) All hydroxides are soluble
(B) All sulfates are soluble
(C) All chlorides, br

In [11]:
prompt = f"""
###Instruction: Act as a science expert and help in answering the following multiple choice question. The answer should be one of the option choices.
Follow the pattern from the first example and answer the second question accordingly.

###Question:
This is a question from high school biology.A piece of potato is dropped into a beaker of pure water.
Which of the following describes the activity after the potato is immersed into the water?
(A) Water moves from the potato into the surrounding water.
(B) Water moves from the surrounding water into the potato.
(C) Potato cells plasmolyze.
(D) Solutes in the water move into the potato.

###Answer:
B

###Question:
You are the world’s best expert in high school biology. From the solubility rules, which of the following is true?
(A) All hydroxides are soluble
(B) All sulfates are soluble
(C) All chlorides, bromides, and iodides are soluble
(D) All ammonium-containing compounds are soluble

###Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
outputs=model(inputs["input_ids"])

next_token_logits = outputs.logits[:, -1]

In [12]:
tokens_of_interest=['A','B','C','D']
token_indices = tokenizer.convert_tokens_to_ids(tokens_of_interest)

In [13]:
indices_in_logits = {token: next_token_logits[0,token_idx].item() for token, token_idx in zip(tokens_of_interest, token_indices)}

In [14]:
indices_in_logits

{'A': 21.578125, 'B': 20.78125, 'C': 21.8125, 'D': 21.5}

In [27]:
def softmax(x):
  e_x=np.exp(x-np.max(x))
  return e_x/e_x.sum()

In [31]:
label_to_softmax_dict=dict(zip(tokens_of_interest,softmax(np.array(list(indices_in_logits.values())))))

In [32]:
label_to_softmax_dict

{'A': 0.274747724454225,
 'B': 0.12383850168956001,
 'C': 0.347313666960872,
 'D': 0.25410010689534285}

In [38]:
df_stress_index=pd.read_csv('data_with_stress_index.csv',usecols=lambda col:col not in ['Unnamed: 0'] )

In [39]:
df_stress_index

,news_sentiment,broker_count_imbalance,volume_indicator,benchmark_price_difference,trade_count_imbalance,one_sided_trade_indicator,tranche_size_indicator,stress_index
0,0.393144,0.124362,0.677889,0.497776,-0.720698,-0.680076,0.724869,0.551515
1,0.772496,0.369376,-0.877238,0.139748,0.851864,-0.257519,0.586063,0.623805
2,-0.189353,0.190013,-0.485352,-0.364914,-0.183833,-0.235899,0.356616,0.305677
3,0.219090,-0.577930,0.876170,0.495462,0.182495,0.094340,0.106455,0.599768
4,0.038766,0.474529,0.345526,-0.153370,-0.020692,0.238055,0.432594,0.594587
...,...,...,...,...,...,...,...,...
4995,-0.502263,0.409759,-0.087573,0.323973,-0.960938,0.359897,0.595025,0.439501
4996,-0.444850,-0.969700,0.050369,-0.105244,0.213531,0.319491,0.455737,0.360712
4997,0.520867,0.304688,-0.802427,0.362181,-0.445552,0.071510,0.163793,0.444237
4998,-0.251193,0.288159,-0.617303,0.445843,0.956045,0.024746,0.470942,0.589725


Bucketing the stress index into 10 different segments (option choices)

In [113]:
[str({i:j for i,j in row.to_dict().items() if i not in ['stress_index','stress_index_bucket']}) for _, row in df_stress_index.head(2).iterrows()]

["{'news_sentiment': 0.3931444486731639, 'broker_count_imbalance': 0.1243619871996783, 'volume_indicator': 0.6778892697073087, 'benchmark_price_difference': 0.4977762417709135, 'trade_count_imbalance': -0.7206982745760815, 'one_sided_trade_indicator': -0.6800763028141159, 'tranche_size_indicator': 0.7248691201811168}",
 "{'news_sentiment': 0.7724956104208642, 'broker_count_imbalance': 0.3693763241525458, 'volume_indicator': -0.8772379830243947, 'benchmark_price_difference': 0.1397481154484952, 'trade_count_imbalance': 0.8518638934191982, 'one_sided_trade_indicator': -0.2575187071551993, 'tranche_size_indicator': 0.5860627411447076}"]

In [56]:
df_stress_index.loc[:,'stress_index_bucket']=pd.cut(df_stress_index['stress_index'],bins=10,labels=[str(np.round(x,1))+'-'+str(np.round(x+0.1,1)) for x in np.arange(0,1,0.1)])

In [57]:
df_stress_index

,news_sentiment,broker_count_imbalance,volume_indicator,benchmark_price_difference,trade_count_imbalance,one_sided_trade_indicator,tranche_size_indicator,stress_index,stress_index_bucket
0,0.393144,0.124362,0.677889,0.497776,-0.720698,-0.680076,0.724869,0.551515,0.5-0.6
1,0.772496,0.369376,-0.877238,0.139748,0.851864,-0.257519,0.586063,0.623805,0.6-0.7
2,-0.189353,0.190013,-0.485352,-0.364914,-0.183833,-0.235899,0.356616,0.305677,0.3-0.4
3,0.219090,-0.577930,0.876170,0.495462,0.182495,0.094340,0.106455,0.599768,0.5-0.6
4,0.038766,0.474529,0.345526,-0.153370,-0.020692,0.238055,0.432594,0.594587,0.5-0.6
...,...,...,...,...,...,...,...,...,...
4995,-0.502263,0.409759,-0.087573,0.323973,-0.960938,0.359897,0.595025,0.439501,0.4-0.5
4996,-0.444850,-0.969700,0.050369,-0.105244,0.213531,0.319491,0.455737,0.360712,0.3-0.4
4997,0.520867,0.304688,-0.802427,0.362181,-0.445552,0.071510,0.163793,0.444237,0.4-0.5
4998,-0.251193,0.288159,-0.617303,0.445843,0.956045,0.024746,0.470942,0.589725,0.5-0.6


**Trading Bot Prompt**

In [99]:
import random
random.seed(42)

In [100]:
def shuffle_list(original_list):
    # Create a copy of the list
    shuffled_list = original_list.copy()

    # Shuffle the copy
    random.shuffle(shuffled_list)

    return shuffled_list

In [101]:
list_stress_index_bucket=[str(np.round(x,1))+'-'+str(np.round(x+0.1,1)) for x in np.arange(0,1,0.1)]
list_option_choices=[chr(x) for x in range(65,65+len(list_stress_index_bucket))]

In [132]:
instruction='''
###Instruction:
You are fixed income corporate bond trader at BlackRock. Your role is to analyse the data that I will provide to you and by analysing the data you need to select the correct option for classifying the stress index (has a range of 0 to 1) into one of the defined buckets.
The stress index is calculated using the features as defined below :

- news_sentiment: that means sentiment of the news published for the bond issuer in last 7 days in a scale of -1 to +1, -1 means negative sentiment about the security and +1 means positive news about the security.
- broker_count_imbalance: that means count difference of brokers trying to buy or sell a security in the same side in the last 2 days. -1 means too many brokers trying to sell the security and +1 means too many brokers trying to buy the security.
- volume_indicator: that means a bond being heavily bought and sold in last 2 days, -1 means a bond being suddenly sold at a very high volume and +1 means a bond being suddenly bought at a very high level.
- benchmark_price_difference: that means how far is the price of a quote for a bond quoted by a broker compared to benchmark price. If a bond is aggressively quoted compared to the benchmark, then it’s a sign of stress for that bond. It is in the range of -1 to +1, -1 means quoted price below the benchmark price and +1 means quoted price is above the benchmark price.
- trade_count_imbalance: that means count difference of actual buy and sell trades of a security in the last 2 days. It is in the range of -1 to +1, -1 means number of sell trades were higher compared to buy trades and +1 means number of buy trades were higher compared to sell trades
- one_sided_trade_indicator: that means count difference of actual buy and sell trades of a security in the same side in the last 7 days. It is in the range of -1 to +1, -1 means too many trades happened at sell side and +1 means too many trades happened in the buy side in the last 7 days
- tranche_size_indicator: If all the trades are in one direction and the amount outstanding of the bond is very high, e.g. $5 billion tranche, then the bond is not stressed, but if the If all the trades are in one direction and the amount outstanding of the bond is very low, e.g. $750 million tranche, then the bond is stressed. It is in the range of 0 to +1, 0 means tranche size is high whereas 1 means tranche size is very low

Note : Act as a finance expert and help in answering the following multiple choice question. The answer should be one of the option choices.
Follow the pattern from the first example and answer the second question accordingly.

###Question:
Given the following feature value dict :
{'news_sentiment': 0.3931444486731639, 'broker_count_imbalance': 0.1243619871996783, 'volume_indicator': 0.6778892697073087, 'benchmark_price_difference': 0.4977762417709135, 'trade_count_imbalance': -0.7206982745760815, 'one_sided_trade_indicator': -0.6800763028141159, 'tranche_size_indicator': 0.7248691201811168}

Return the correct option for classifying the stress index from the following options based on the features given above. Do not generate anything else apart from the option.
'''

In [133]:
choice_prompt=''''''
for i,j in zip(list_option_choices,shuffle_list(list_stress_index_bucket)):
  choice_prompt+=f'({i}) {j}\n'

In [134]:
example_prompt=''''''
example_prompt+=instruction
example_prompt+=choice_prompt
example_prompt+='''
###Answer: E
'''

In [135]:
print(example_prompt)


###Instruction: 
You are fixed income corporate bond trader at BlackRock. Your role is to analyse the data that I will provide to you and by analysing the data you need to select the correct option for classifying the stress index (has a range of 0 to 1) into one of the defined buckets.
The stress index is calculated using the features as defined below : 
 
- news_sentiment: that means sentiment of the news published for the bond issuer in last 7 days in a scale of -1 to +1, -1 means negative sentiment about the security and +1 means positive news about the security.
- broker_count_imbalance: that means count difference of brokers trying to buy or sell a security in the same side in the last 2 days. -1 means too many brokers trying to sell the security and +1 means too many brokers trying to buy the security.
- volume_indicator: that means a bond being heavily bought and sold in last 2 days, -1 means a bond being suddenly sold at a very high volume and +1 means a bond being suddenly b

In [136]:
final_prompt=''''''
final_prompt+=example_prompt
final_prompt+='''
###Question:
Given the following feature value dict :
{'news_sentiment': 0.7724956104208642, 'broker_count_imbalance': 0.3693763241525458, 'volume_indicator': -0.8772379830243947, 'benchmark_price_difference': 0.1397481154484952, 'trade_count_imbalance': 0.8518638934191982, 'one_sided_trade_indicator': -0.2575187071551993, 'tranche_size_indicator': 0.5860627411447076}

Return the correct option for classifying the stress index from the following options based on the features given above. Do not generate anything else apart from the option.
'''
choice_prompt=''''''
for i,j in zip(list_option_choices,shuffle_list(list_stress_index_bucket)):
  choice_prompt+=f'({i}) {j}\n'

final_prompt+=choice_prompt
final_prompt+='''
###Answer:
'''

In [137]:
print(final_prompt)


###Instruction: 
You are fixed income corporate bond trader at BlackRock. Your role is to analyse the data that I will provide to you and by analysing the data you need to select the correct option for classifying the stress index (has a range of 0 to 1) into one of the defined buckets.
The stress index is calculated using the features as defined below : 
 
- news_sentiment: that means sentiment of the news published for the bond issuer in last 7 days in a scale of -1 to +1, -1 means negative sentiment about the security and +1 means positive news about the security.
- broker_count_imbalance: that means count difference of brokers trying to buy or sell a security in the same side in the last 2 days. -1 means too many brokers trying to sell the security and +1 means too many brokers trying to buy the security.
- volume_indicator: that means a bond being heavily bought and sold in last 2 days, -1 means a bond being suddenly sold at a very high volume and +1 means a bond being suddenly b

In [138]:
inputs = tokenizer(final_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        temperature=0.2,
        do_sample=True,
        max_new_tokens=16
        # stopping_criteria = [EosListStoppingCriteria()]
    )[0],
    skip_special_tokens=False
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
<s> 
###Instruction: 
You are fixed income corporate bond trader at BlackRock. Your role is to analyse the data that I will provide to you and by analysing the data you need to select the correct option for classifying the stress index (has a range of 0 to 1) into one of the defined buckets.
The stress index is calculated using the features as defined below : 
 
- news_sentiment: that means sentiment of the news published for the bond issuer in last 7 days in a scale of -1 to +1, -1 means negative sentiment about the security and +1 means positive news about the security.
- broker_count_imbalance: that means count difference of brokers trying to buy or sell a security in the same side in the last 2 days. -1 means too many brokers trying to sell the security and +1 means too many brokers trying to buy the security.
- volume_indicator: that means a bond being h